Importando Pacotes

In [24]:
#!pip install -r requirements.txt

In [25]:
import pandas as pd
import numpy as np

Configurações do código

In [26]:
#Exibir totas as colunas no pandas
pd.set_option('display.max_columns', None)
season = 2023

### Partidas

In [27]:
df_partidas_raw = pd.read_parquet(f'data/raw/brasileirao/partidas/partidas_{season}.parquet')

In [28]:
df_partidas_raw.head()

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,1005649,Rodrigo José Pereira de Lima,America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1681671600,1681675200,NaN,Estadio Jornalista Mário Filho,"Rio de Janeiro, Rio de Janeiro",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,127,Flamengo,https://media-4.api-sports.io/football/teams/1...,True,147,Coritiba,https://media-4.api-sports.io/football/teams/1...,False,3,0,1,0,3,0,None,None,None,None
1,1005650,Ramon Abatti,America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1681594200,1681597800,218.0,Estádio Nilton Santos,Rio de Janeiro,Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,120,Botafogo,https://media-4.api-sports.io/football/teams/1...,True,126,Sao Paulo,https://media-4.api-sports.io/football/teams/1...,False,2,1,1,1,2,1,None,None,None,None
2,1005651,Paulo Cesar Zanovelli da Silva,America/Sao_Paulo,2023-04-15 16:00:00-03:00,1681585200,1681585200,1681588800,258.0,Allianz Parque,"São Paulo, São Paulo",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,121,Palmeiras,https://media-4.api-sports.io/football/teams/1...,True,1193,Cuiaba,https://media-4.api-sports.io/football/teams/1...,False,2,1,1,1,2,1,None,None,None,None
3,1005652,Anderson Daronco,America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1681671600,1681675200,11531.0,Neo Química Arena,"São Paulo, São Paulo",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,131,Corinthians,https://media-4.api-sports.io/football/teams/1...,True,135,Cruzeiro,https://media-4.api-sports.io/football/teams/1...,False,2,1,0,0,2,1,None,None,None,None
4,1005653,Maguielson Lima Barbosa,America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1681594200,1681597800,220.0,Estádio Nabi Abi Chedid,"Bragança Paulista, São Paulo",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,794,RB Bragantino,https://media-4.api-sports.io/football/teams/7...,True,118,Bahia,https://media-4.api-sports.io/football/teams/1...,False,2,1,0,1,2,1,None,None,None,None


In [29]:
df_partidas = df_partidas_raw.copy()

df_partidas['fixture.date'] = df_partidas['fixture.date'].dt.date
df_partidas['league.round'] = df_partidas['league.round'].replace('Regular Season - ', '', regex=True)

df_partidas = df_partidas[['fixture.id', 'fixture.date', 'league.round',
                           'teams.home.id', 'teams.home.name', 'teams.home.winner', 
                           'teams.away.id', 'teams.away.name', 'teams.away.winner',
                           'score.halftime.home', 'score.halftime.away',
                           'score.fulltime.home', 'score.fulltime.away']]

df_partidas = df_partidas.astype({'league.round':'int',
                                  'teams.home.winner': 'bool',
                                  'teams.away.winner': 'bool'})

In [30]:
df_partidas.head()

,fixture.id,fixture.date,league.round,teams.home.id,teams.home.name,teams.home.winner,teams.away.id,teams.away.name,teams.away.winner,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away
0,1005649,2023-04-16,1,127,Flamengo,True,147,Coritiba,False,1,0,3,0
1,1005650,2023-04-15,1,120,Botafogo,True,126,Sao Paulo,False,1,1,2,1
2,1005651,2023-04-15,1,121,Palmeiras,True,1193,Cuiaba,False,1,1,2,1
3,1005652,2023-04-16,1,131,Corinthians,True,135,Cruzeiro,False,0,0,2,1
4,1005653,2023-04-15,1,794,RB Bragantino,True,118,Bahia,False,0,1,2,1


Coluna de Resultado do Jogo  
1=Mandante | 2=Visitante | 0=Empate

In [31]:
df_partidas['result'] = \
    np.where( df_partidas['teams.home.winner'], 1,
    np.where( df_partidas['teams.away.winner'], 2,        
    0))

df_partidas['result.team'] = \
    np.where( df_partidas['teams.home.winner'], df_partidas['teams.home.name'],
    np.where( df_partidas['teams.away.winner'], df_partidas['teams.away.name'],      
    'Empate'))


#Half Time Result
df_partidas['HTR'] = \
    np.where( df_partidas['score.halftime.home'] > df_partidas['score.halftime.away'], 1,
    np.where( df_partidas['score.halftime.home'] < df_partidas['score.halftime.away'], 2,        
    0))

### Performance últimos jogos
Calcula taxa de Empate, Vitoria, Derrota para Home e Away

In [33]:
def calculo_aproveitamento(row):
    
    periodo = 3 #avalia ultimos 3 jogos em casa e visitante
    round = row['league.round']
    home_id = row['teams.home.id']
    away_id = row['teams.away.id']

    if round < 3:
        return None, None, None, None, None, None

    base_home = df_partidas.loc[
            (df_partidas['teams.home.id'] == home_id) & 
            (df_partidas['league.round'] < round)
        ].sort_values('league.round').tail(periodo)
    

    base_away = df_partidas.loc[
            (df_partidas['teams.away.id'] == away_id) & 
            (df_partidas['league.round'] < round)
        ].sort_values('league.round').tail(periodo)

    home_matches = len(base_home)
    home_drw = len(base_home.loc[base_home['result'] == 0]) / home_matches
    home_win = len(base_home.loc[base_home['result'] == 1]) / home_matches
    home_los = len(base_home.loc[base_home['result'] == 2]) / home_matches

    away_matches = len(base_away)
    away_drw = len(base_away.loc[base_away['result'] == 0]) / away_matches
    away_win = len(base_away.loc[base_away['result'] == 2]) /away_matches
    away_los = len(base_away.loc[base_away['result'] == 1]) / away_matches

    return home_drw, home_win, home_los, \
           away_drw, away_win, away_los

In [34]:
df_partidas[['home.pf.drw', 'home.pf.win', 'home.pf.los',
             'away.pf.drw', 'away.pf.win', 'away.pf.los']] = \
    df_partidas.apply(lambda x: calculo_aproveitamento(x), axis=1, result_type='expand')\
               .round(2)

### Estatisticas da Partida

In [291]:
df_partidas_stats_raw = pd.read_parquet(f'data/raw/brasileirao/partidas_estatistica/{season}')

In [292]:
df_partidas_stats_raw.head()

,type,value,team.id,team.name,fixture.id
0,Shots on Goal,7,127,Flamengo,1005649
1,Shots off Goal,5,127,Flamengo,1005649
2,Total Shots,15,127,Flamengo,1005649
3,Blocked Shots,3,127,Flamengo,1005649
4,Shots insidebox,5,127,Flamengo,1005649


Renomeando Variaveis para incurtar no modelo

In [293]:
dict_rename_type = {
    'Shots on Goal': 'SHOG',
    'Shots off Goal': 'SHFG',
    'Total Shots' : 'TSH',
    'Blocked Shots' : 'BSH',
    'Shots insidebox' : 'SHIN',
    'Shots outsidebox' : 'SHOU',
    'Fouls' : 'FOU',
    'Corner Kicks' : 'CK',
    'Offsides' : 'OFSD',
    'Ball Possession' : 'BLPS',
    'Yellow Cards': 'YCD',
    'Red Cards' : 'RCD',
    'Goalkeeper Saves' : 'GKSV',
    'Total passes' : 'TPAS',
    'Passes accurate': 'PASAC',
    'Passes %' : 'PAS%',
    'expected_goals' : 'EXPGL'
}

In [294]:
df_partidas_stats = df_partidas_stats_raw.copy()

df_partidas_stats['type'] = df_partidas_stats['type'].replace(dict_rename_type)
df_partidas_stats['value'] = df_partidas_stats['value'].replace('%', '', regex=True)
df_partidas_stats['value'] = df_partidas_stats['value'].replace('None', 0, regex=True)
df_partidas_stats['value'] = df_partidas_stats['value'].astype('float')

df_partidas_stats.loc[
    df_partidas_stats['type'].isin(['Ball Possession', 'Passes %']), 'value'
] = df_partidas_stats['value']/100

In [295]:
df_stats = pd.pivot(df_partidas_stats, index=['fixture.id', 'team.name'], columns='type', values='value').reset_index()

In [296]:
df_stats.head()

type,fixture.id,team.name,BLPS,BSH,CK,EXPGL,FOU,GKSV,OFSD,PAS%,PASAC,RCD,SHFG,SHIN,SHOG,SHOU,TPAS,TSH,YCD
0,1005649,Coritiba,41.0,5.0,3.0,0.36,17.0,4.0,2.0,85.0,322.0,0.0,2.0,3.0,3.0,7.0,379.0,10.0,3.0
1,1005649,Flamengo,59.0,3.0,6.0,1.37,20.0,3.0,1.0,89.0,480.0,0.0,5.0,5.0,7.0,10.0,542.0,15.0,3.0
2,1005650,Botafogo,32.0,4.0,3.0,0.84,20.0,7.0,2.0,78.0,229.0,0.0,5.0,6.0,3.0,6.0,293.0,12.0,3.0
3,1005650,Sao Paulo,68.0,2.0,7.0,1.86,13.0,1.0,1.0,89.0,574.0,0.0,6.0,11.0,8.0,5.0,646.0,16.0,2.0
4,1005651,Cuiaba,48.0,2.0,4.0,0.67,12.0,3.0,2.0,79.0,337.0,1.0,6.0,6.0,4.0,6.0,426.0,12.0,5.0


### Base Tratada

In [297]:
df_base_final = \
    df_partidas.merge(
        df_stats.add_prefix('home.'),
        how='inner',
        left_on=['fixture.id', 'teams.home.name'],
        right_on=['home.fixture.id', 'home.team.name']
    )\
    .merge(
        df_stats.add_prefix('away.'),
        how='inner',
        left_on=['fixture.id', 'teams.away.name'],
        right_on=['away.fixture.id', 'away.team.name']
    )

df_base_final = df_base_final.drop(columns=['home.fixture.id', 'home.team.name',
                                            'away.fixture.id', 'away.team.name'])



In [298]:
df_base_final.head()

,fixture.id,fixture.date,league.round,teams.home.id,teams.home.name,teams.home.winner,teams.away.id,teams.away.name,teams.away.winner,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,result,result.team,HTR,home.pf.drw,home.pf.win,home.pf.los,away.pf.drw,away.pf.win,away.pf.los,home.BLPS,home.BSH,home.CK,home.EXPGL,home.FOU,home.GKSV,home.OFSD,home.PAS%,home.PASAC,home.RCD,home.SHFG,home.SHIN,home.SHOG,home.SHOU,home.TPAS,home.TSH,home.YCD,away.BLPS,away.BSH,away.CK,away.EXPGL,away.FOU,away.GKSV,away.OFSD,away.PAS%,away.PASAC,away.RCD,away.SHFG,away.SHIN,away.SHOG,away.SHOU,away.TPAS,away.TSH,away.YCD
0,1005649,2023-04-16,1,127,Flamengo,True,147,Coritiba,False,1,0,3,0,1,Flamengo,1,NaN,NaN,NaN,NaN,NaN,NaN,59.0,3.0,6.0,1.37,20.0,3.0,1.0,89.0,480.0,0.0,5.0,5.0,7.0,10.0,542.0,15.0,3.0,41.0,5.0,3.0,0.36,17.0,4.0,2.0,85.0,322.0,0.0,2.0,3.0,3.0,7.0,379.0,10.0,3.0
1,1005650,2023-04-15,1,120,Botafogo,True,126,Sao Paulo,False,1,1,2,1,1,Botafogo,0,NaN,NaN,NaN,NaN,NaN,NaN,32.0,4.0,3.0,0.84,20.0,7.0,2.0,78.0,229.0,0.0,5.0,6.0,3.0,6.0,293.0,12.0,3.0,68.0,2.0,7.0,1.86,13.0,1.0,1.0,89.0,574.0,0.0,6.0,11.0,8.0,5.0,646.0,16.0,2.0
2,1005651,2023-04-15,1,121,Palmeiras,True,1193,Cuiaba,False,1,1,2,1,1,Palmeiras,0,NaN,NaN,NaN,NaN,NaN,NaN,52.0,5.0,4.0,2.69,23.0,3.0,3.0,84.0,382.0,0.0,7.0,10.0,5.0,7.0,455.0,17.0,2.0,48.0,2.0,4.0,0.67,12.0,3.0,2.0,79.0,337.0,1.0,6.0,6.0,4.0,6.0,426.0,12.0,5.0
3,1005652,2023-04-16,1,131,Corinthians,True,135,Cruzeiro,False,0,0,2,1,1,Corinthians,0,NaN,NaN,NaN,NaN,NaN,NaN,42.0,1.0,7.0,1.54,14.0,1.0,4.0,78.0,316.0,0.0,4.0,6.0,7.0,6.0,406.0,12.0,4.0,58.0,3.0,4.0,0.49,12.0,5.0,3.0,85.0,489.0,0.0,3.0,4.0,2.0,4.0,572.0,8.0,2.0
4,1005653,2023-04-15,1,794,RB Bragantino,True,118,Bahia,False,0,1,2,1,1,RB Bragantino,2,NaN,NaN,NaN,NaN,NaN,NaN,56.0,2.0,4.0,2.11,20.0,2.0,2.0,78.0,339.0,0.0,4.0,6.0,7.0,7.0,436.0,13.0,3.0,44.0,1.0,7.0,0.65,13.0,5.0,3.0,72.0,248.0,0.0,5.0,5.0,3.0,4.0,343.0,9.0,1.0


In [299]:
df_base_final.to_excel(f'data/trusted/base_brasileirao_{season}.xlsx', index=False)